# Final Project: Rijksmuseum Artist Classification Task
## CSC 594: Advanced Deep Learning
## Ross Pollock

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras 
import tensorflow.keras.layers as L

import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
# Extract Data to local colab drive
! tar xfz /drive/My\ Drive/CSC594/Data/Image/Rijksmuseum.tar.gz -C /content/

In [ ]:


'''
Implementation of ResNet 50,101,152
with optional width parameter for 
making wider ResNets
'''

class ResNetBlock(L.Layer):
  def __init__(
      self,n_filters,regularizer=keras.regularizers.L2(1e-3),
      activation=L.ReLU,start=False,downsample=False
    ):
    super(ResNetBlock,self).__init__()
    self.regularizer = regularizer
    self.activation = activation
    self.n_filters = n_filters
    self.start = start
    self.stride = 2 if downsample else 1


    self.left_path = keras.Sequential(
        [
          L.Conv2D(self.n_filters,1,self.stride,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters,3,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters*4,1,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization()
        ]
    )
  
    # If first block add convolution to residual path
    # so that the number of output channels match
    if self.start:
      self.right_path = keras.Sequential(
          [
           L.Conv2D(self.n_filters*4,1,self.stride,'same',kernel_regularizer=self.regularizer),
           L.BatchNormalization()
          ]
      )

    

  def call(self,input_tensor,training=False):
    x = self.left_path(input_tensor)
    if self.start:
      y = self.right_path(input_tensor)
    else:
      y = input_tensor

    z = tf.add(x,y)
    return self.activation()(z)
      

class ResNetStack(L.Layer):
  def __init__(self,
      n_blocks,n_filters,
      first_block=False,**kwargs
    ):
    super(ResNetStack,self).__init__()
    blocks = [ 
      ResNetBlock(n_filters,start=(not i),downsample=(not i and not first_block),**kwargs) 
      for i in range(n_blocks)
    ]
    self.stack = keras.Sequential(
        blocks
    )
    self.out_dim = n_filters * 4

  def call(self,input_tensor,training=False):
    return self.stack(input_tensor) 

def ResNet(n_layers,width=1,input_shape=None,**kwargs):
    if n_layers not in [50,101,152]:
      raise ValueError
    
    model = keras.Sequential([
        L.Input(input_shape),
        L.Conv2D(64,7,2,padding='same'),
        L.BatchNormalization(),
        L.ReLU(),
        L.MaxPool2D(3,2,'same')
        
    ],name=f'ResNet{n_layers}')


    model_specs = {
        50: [3,4,6,3],
        101: [3,4,23,3],
        152: [3,8,36,3]
    }
    filters = list(map(lambda x: x*width,[64,128,256,512]))

    for i,(stack,f) in enumerate(zip(model_specs[n_layers],filters)):
      if i == 0:
        stack = ResNetStack(stack,f,first_block=True)
      else:
        stack = ResNetStack(stack,f,first_block=False)
      model.add(stack)
    model.add(L.GlobalAveragePooling2D())
    return model




## Load Pre-trained Model

In [ ]:
model_url = 'https://rosspollock.design/models/resnet50x2_120k.tar.gz'
keras.utils.get_file('model',model_url,untar=True,extract=True)
encoder = keras.models.load_model('/root/.keras/datasets/query_encoder120k')
encoder = encoder.get_layer('ResNet50')
if encoder.name == 'ResNet50':
  new_encoder = ResNet(50,2,input_shape=(244,244,3))
  new_encoder.set_weights(encoder.get_weights())
  encoder = new_encoder 
  del new_encoder
_, rep_dim = encoder.output_shape

375029760/375028329 [==============================] - 7s 0us/step


In [ ]:
@tf.function
def smart_resize(img,size=244):
  h,w = tf.shape(img)[0], tf.shape(img)[1]
  min_dim = tf.minimum(h,w)
  ratio = size / tf.cast(min_dim,tf.float32) 

  new_w = tf.cast(w,tf.float32) * ratio
  new_h = tf.cast(h,tf.float32) * ratio

  img = tf.image.resize(img,[new_h,new_w],preserve_aspect_ratio=True)
  img = tf.image.resize_with_crop_or_pad(img,size,size)
  return img

@tf.function
def img_preprocess(fpath):
    img = tf.io.read_file(fpath)
    img = tf.image.decode_jpeg(img,3)
    img = tf.cast(img,tf.float32)
    img = smart_resize(img,size=244)
    img /= (255/2)
    img -= 1 
    return img 

## Load Image Paths and Labels

In [ ]:
test_df = pd.read_csv('/drive/My Drive/CSC594/Data/Rijkmuseum_test_metadata.csv')
filter_test = test_df[(test_df.object_creator != 'anonymous') & (test_df.object_creator != 'unknown')]
filter_test = filter_test.groupby('object_creator').filter(lambda x: x['object_path'].count() > 10)

train_df = pd.read_csv('/drive/My Drive/CSC594/Data/Rijkmuseum_train_metadata.csv')
filter_train = train_df[train_df.object_creator.isin(filter_test.object_creator)]

n_artist, _ = filter_test.groupby('object_creator').count().shape

In [ ]:
train_paths = filter_train.object_path.map(lambda x: '/content/smaller/'+x).to_list()
test_paths = filter_test.object_path.map(lambda x: '/content/smaller/'+x).to_list()

In [ ]:
y_train = filter_train.object_creator
y_test = filter_test.object_creator

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
n_train = y_train.shape[0]
n_test = y_test.shape[0]

## Extract Representations 

In [ ]:
x_train = np.zeros((n_train,rep_dim))
x_test = np.zeros((n_test,rep_dim))

In [ ]:
BATCH_SIZE = 128
ds_train = tf.data.Dataset.from_tensor_slices(train_paths)
ds_train = ds_train.map(img_preprocess)
ds_train = ds_train.batch(BATCH_SIZE)

ds_test = tf.data.Dataset.from_tensor_slices(test_paths)
ds_test = ds_test.map(img_preprocess)
ds_test = ds_test.batch(BATCH_SIZE)

In [ ]:
pbar = keras.utils.Progbar(n_train // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_train):
  reps = encoder(batch,training=False)
  x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

pbar = keras.utils.Progbar(n_test // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_test):
  reps = encoder(batch,training=False)
  x_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

97/97 [==============================] - 124s 1s/step


## Fit Classifier

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
clf = keras.Sequential([
  L.Dense(label_encoder.classes_.shape[0],'softmax')
])


In [ ]:
clf.compile(
    keras.optimizers.Adam(),
    keras.losses.SparseCategoricalCrossentropy(),
    keras.metrics.SparseCategoricalAccuracy()
)

In [ ]:
clf.fit(x=x_train,y=y_train,epochs=50,validation_split=0.2)

Epoch 1/50
1198/1198 [==============================] - 4s 3ms/step - loss: 3.7891 - sparse_categorical_accuracy: 0.2470 - val_loss: 3.2016 - val_sparse_categorical_accuracy: 0.3331
Epoch 2/50
1198/1198 [==============================] - 3s 3ms/step - loss: 2.9202 - sparse_categorical_accuracy: 0.3727 - val_loss: 2.9049 - val_sparse_categorical_accuracy: 0.3895
Epoch 3/50
1198/1198 [==============================] - 3s 3ms/step - loss: 2.6408 - sparse_categorical_accuracy: 0.4192 - val_loss: 2.7732 - val_sparse_categorical_accuracy: 0.4126
Epoch 4/50
1198/1198 [==============================] - 3s 3ms/step - loss: 2.4756 - sparse_categorical_accuracy: 0.4481 - val_loss: 2.6835 - val_sparse_categorical_accuracy: 0.4312
Epoch 5/50
1198/1198 [==============================] - 3s 3ms/step - loss: 2.3600 - sparse_categorical_accuracy: 0.4710 - val_loss: 2.6352 - val_sparse_categorical_accuracy: 0.4359
Epoch 6/50
1198/1198 [==============================] - 3s 3ms/step - loss: 2.2715 - spars

In [ ]:
test_loss, test_acc = clf.evaluate(x_test,y_test)
print(f'Test Accuracy {100*test_acc:0.2f}')

386/386 [==============================] - 1s 2ms/step - loss: 2.6020 - sparse_categorical_accuracy: 0.4734
Test Accuracy 47.34
